In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# K-matrix

While {mod}`ampform` does not yet provide a generic way to produce an amplitude model with $\boldsymbol{K}$-matrix dynamics, the (experimental) {mod}`.kmatrix` module makes fairly simple to produce an expression for the $\boldsymbol{K}$-matrix and play around with it interactively. For more info on the $\boldsymbol{K}$-matrix, see this instructive presentation {cite}`meyerMatrixTutorial2008`, or the classic paper {cite}`chungPartialWaveAnalysis1995`.

In [ ]:
%matplotlib widget
import logging
import re
import warnings

import ipywidgets
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import sympy as sp
from IPython.display import Image
from matplotlib import cm
from mpl_interactions.controller import Controls

import symplot
from ampform.dynamics import PhaseSpaceFactor, PhaseSpaceFactorComplex
from ampform.dynamics.kmatrix import KMatrix

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

## Non-relativistic

::::{margin}
:::{seealso} {doc}`compwa-org:report/005`
:::
::::

This section shows how a $\boldsymbol{K}$-matrix for **one channel with two poles** compares to using a sum of two Breit-Wigner functions.

A non-relativistic $\boldsymbol{K}$-matrix for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.NonRelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import NonRelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
k_matrix_nr = NonRelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
k_matrix_nr[0, 0]

Notice how the $\boldsymbol{K}$-matrix reduces to a {func}`.relativistic_breit_wigner` when there is one channel and one resonance (but for a residue constant $\gamma$):

In [ ]:
k_matrix_1r = NonRelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
k_matrix_1r[0, 0].doit().simplify()

Now let's investigate the effect of using a $\boldsymbol{K}$-matrix to describe **two resonances** in one channel and see how it compares with the sum of two Breit-Wigner functions. Two Breit-Wigner 'poles' with the same parameters would look like this:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner

s, m1, m2, Gamma1, Gamma2, gamma1, gamma2 = sp.symbols(
    "s, m1, m2, Gamma1, Gamma2, gamma1, gamma2"
)
term1 = relativistic_breit_wigner(s, m1, Gamma1)
term2 = relativistic_breit_wigner(s, m2, Gamma2)
bw = gamma1 * term1 + gamma2 * term2
bw

while a $\boldsymbol{K}$-matrix parametrizes the two resonances as:

In [ ]:
k_matrix_2r = NonRelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
k_matrix = k_matrix_2r[0, 0].doit()

In [ ]:
# reformulate terms
denominator, nominator = k_matrix.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
k_matrix = term1 + term2
k_matrix

To simplify things, we can set the residue constants $\gamma$ to one. Notice how the $\boldsymbol{K}$-matrix has introduced some coupling ('interference') between the two terms.

In [ ]:
def remove_residue_constants(expression):
    expression = symplot.substitute_indexed_symbols(expression)
    residue_constants = filter(
        lambda s: re.match(r"^\\?gamma", s.name),
        expression.free_symbols,
    )
    return expression.xreplace({gamma: 1 for gamma in residue_constants})


display(
    remove_residue_constants(bw),
    remove_residue_constants(k_matrix),
)

Now, just like in {doc}`/usage/interactive`, we use {mod}`symplot` to visualize the difference between the two expressions. The important thing is that the Argand plot on the right shows that **the $\boldsymbol{K}$-matrix conserves unitarity**.

Note that we have to call {func}`symplot.substitute_indexed_symbols` to turn the {class}`~sympy.tensor.indexed.Indexed` instances in this {obj}`~sympy.matrices.dense.Matrix` expression into {class}`~sympy.core.symbol.Symbol`s before calling this function. We also call {func}`symplot.rename_symbols` so that the residue $\gamma$'s get a name that does not have to be dummified by {func}`~sympy.utilities.lambdify.lambdify`.

In [ ]:
# Prepare expressions
m = sp.Symbol("m")
kmatrix = symplot.substitute_indexed_symbols(k_matrix)
rename_gamma = lambda s: re.sub(  # noqa: E731
    r"\\gamma_{([0-9]),0}", r"gamma\1", s
)
bw = symplot.rename_symbols(bw, rename_gamma)
kmatrix = symplot.rename_symbols(kmatrix, rename_gamma)
bw = bw.xreplace({s: m ** 2})
kmatrix = kmatrix.xreplace({s: m ** 2})

# Prepare sliders and domain
np_kmatrix, sliders = symplot.prepare_sliders(kmatrix, m)
np_bw = sp.lambdify((m, Gamma1, Gamma2, gamma1, gamma2, m1, m2), bw.doit())

m_min, m_max = 0, 3
domain_1d = np.linspace(m_min, m_max, 200)
domain_argand = np.linspace(m_min - 2, m_max + 2, 1_000)
sliders.set_ranges(
    m1=(0, 3, 100),
    m2=(0, 3, 100),
    Gamma1=(0, 2, 100),
    Gamma2=(0, 2, 100),
    gamma1=(0, 2),
    gamma2=(0, 2),
)
sliders.set_values(
    m1=1.1,
    m2=1.9,
    Gamma1=0.2,
    Gamma2=0.3,
    gamma1=1,
    gamma2=1,
)
if STATIC_WEB_PAGE:
    # Concatenate flipped domain for reverse animation
    domain = np.linspace(1.0, 2.7, 50)
    domain = np.concatenate((domain, np.flip(domain[1:])))
    sliders._sliders["m1"] = domain


def create_argand(func):
    def wrapped(**kwargs):
        values = func(domain_argand, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return wrapped


# Create figure
fig, axes = plt.subplots(
    ncols=2,
    figsize=1.2 * np.array((8, 3.8)),
    tight_layout=True,
)
ax_intensity, ax_argand = axes
m_label = "$m_{a+b}$"
ax_intensity.set_xlabel(m_label)
ax_intensity.set_ylabel("$|A|^2$")
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")

# Plot intensity
controls = iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_kmatrix(*args, **kwargs) ** 2),
    label="$K$-matrix",
    **sliders,
    ylim="auto",
    ax=ax_intensity,
)
iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_bw(*args, **kwargs) ** 2),
    label="Breit-Wigner",
    controls=controls,
    ylim="auto",
    ax=ax_intensity,
)
plt.legend(loc="upper right")
iplt.axvline(controls["m1"], c="gray", linestyle="dotted")
iplt.axvline(controls["m2"], c="gray", linestyle="dotted")

# Argand plots
iplt.scatter(
    create_argand(np_kmatrix),
    label="$K$-matrix",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
iplt.scatter(
    create_argand(np_bw),
    label="Breit-Wigner",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
plt.legend(loc="upper right");

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "non-relativistic-k-matrix.gif"
    ax_intensity.set_ylim([0, 2])
    ax_argand.set_xlim([-1, +1])
    ax_argand.set_ylim([0, 2])
    controls.save_animation(output_path, fig, "m1", fps=20)
    with open(output_path, "rb") as f:
        display(Image(data=f.read(), format="png"))

## Relativistic

::::{margin}
:::{seealso} {doc}`compwa-org:report/009`
:::
::::

Relativistic $\boldsymbol{K}$-matrices for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.RelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import RelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
rel_k_matrix_nr = RelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
rel_k_matrix_nr[0, 0]

Again, as in {ref}`usage/dynamics/k-matrix:Non-relativistic`, the $\boldsymbol{K}$-matrix reduces to something of a {func}`.relativistic_breit_wigner`. This time, the width has been replaced by a {class}`.CoupledWidth` and some {class}`.PhaseSpaceFactor`s have been inserted that take case of the decay into two decay products:[^1]

[^1]: The two {class}`.PhaseSpaceFactor`s $\sqrt{\rho_0(s)}$ cancel out within phase space, where they are real.

In [ ]:
rel_k_matrix_1r = RelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
symplot.partial_doit(rel_k_matrix_1r[0, 0], sp.Sum).simplify(doit=False)

Similarly, the $\boldsymbol{K}$-matrix with two resonances becomes (neglecting the $\sqrt{\rho_0(s)}$):

In [ ]:
rel_k_matrix_2r = RelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
rel_k_matrix_2r = symplot.partial_doit(rel_k_matrix_2r[0, 0], sp.Sum)

In [ ]:
rel_k_matrix_2r = symplot.substitute_indexed_symbols(rel_k_matrix_2r)
s, m_a, m_b = sp.symbols("s, m_a0, m_b0")
rho = PhaseSpaceFactor(s, m_a, m_b)
rel_k_matrix_2r = rel_k_matrix_2r.xreplace(
    {
        sp.sqrt(rho): 1,
        sp.conjugate(sp.sqrt(rho)): 1,
    }
)
denominator, nominator = rel_k_matrix_2r.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
rel_k_matrix_2r = term1 + term2
rel_k_matrix_2r

## Interactive visualization

The $\boldsymbol{K}$-matrix for arbitrary resonances and channels can be interactively inspected with the applet:

In [ ]:
L = sp.Symbol("L", integer=True, negative=False)
if STATIC_WEB_PAGE:
    L = 0


def plot(
    kmatrix_type: KMatrix,
    n_channels: int,
    n_resonances: int,
    angular_momentum: sp.Symbol = 0,
) -> None:
    # Convert to Symbol: symplot cannot handle IndexedBase
    i, j = sp.symbols("i, j", integer=True, negative=False)
    j = i
    expr = kmatrix_type.formulate(
        n_resonances=n_resonances,
        n_channels=n_channels,
        angular_momentum=angular_momentum,
        phsp_factor=PhaseSpaceFactorComplex,
    ).doit()[i, j]
    expr = expr.xreplace({s: m ** 2})
    expr = symplot.substitute_indexed_symbols(expr)
    np_expr, sliders = symplot.prepare_sliders(expr, m)
    symbol_to_arg = {
        symbol: arg for arg, symbol in sliders._arg_to_symbol.items()
    }

    # Set plot domain
    x_min, x_max = 1e-3, 3
    y_min, y_max = -0.5, +0.5

    plot_domain = np.linspace(x_min, x_max, num=500, dtype=np.complex)
    x_values = np.linspace(x_min, x_max, num=160)
    y_values = np.linspace(y_min, y_max, num=80)
    X, Y = np.meshgrid(x_values, y_values)
    plot_domain_complex = X + Y * 1j

    # Set slider values and ranges
    m0_values = np.linspace(x_min, x_max, num=n_resonances + 2)
    m0_values = m0_values[1:-1]

    def set_default_values():
        if "L" in sliders:
            sliders.set_ranges(L=(0, 8))
        sliders.set_ranges(i=(0, n_channels - 1))
        for R in range(1, n_resonances + 1):
            for i in range(n_channels):
                if kmatrix_type is RelativisticKMatrix:
                    sliders.set_ranges(
                        {
                            f"m{R}": (0, 3, 100),
                            fR"\Gamma_{{{R},{i}}}": (-2, +2, 100),
                            fR"\gamma_{{{R},{i}}}": (0, 10, 100),
                            f"m_a{i}": (0, 1, 0.01),
                            f"m_b{i}": (0, 1, 0.01),
                        }
                    )
                    sliders.set_values(
                        {
                            f"m{R}": m0_values[R - 1],
                            fR"\Gamma_{{{R},{i}}}": 2.0
                            * (0.4 + R * 0.2 - i * 0.3),
                            fR"\gamma_{{{R},{i}}}": 0.25 * (10 - R + i),
                            f"m_a{i}": (i + 1) * 0.25,
                            f"m_b{i}": (i + 1) * 0.25,
                        }
                    )
                if kmatrix_type is NonRelativisticKMatrix:
                    sliders.set_ranges(
                        {
                            f"m{R}": (0, 3, 100),
                            f"Gamma{R}": (-1, 1, 100),
                            fR"\gamma_{{{R},{i}}}": (0, 2, 100),
                        }
                    )
                    sliders.set_values(
                        {
                            f"m{R}": m0_values[R - 1],
                            f"Gamma{R}": (R + 1) * 0.1,
                            fR"\gamma_{{{R},{i}}}": 1 - 0.1 * R + 0.1 * i,
                        }
                    )

    set_default_values()

    # Create interactive plots
    controls = Controls(**sliders)
    fig, axes = plt.subplots(
        nrows=2,
        figsize=(8, 6),
        sharex=True,
        tight_layout=True,
    )
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.suptitle(
        fR"${n_channels} \times {n_channels}$ $K$-matrix"
        f" with {n_resonances} resonances"
    )

    for ax in axes:
        ax.set_xlim(x_min, x_max)
    ax_2d, ax_3d = axes
    ax_2d.set_ylabel("$|T|^{2}$")
    ax_2d.set_yticks([])
    ax_3d.set_xlabel("Re $m$")
    ax_3d.set_ylabel("Im $m$")
    ax_3d.set_xticks([])
    ax_3d.set_yticks([])
    ax_3d.set_facecolor("white")

    ax_3d.axhline(0, linewidth=0.5, c="black", linestyle="dotted")

    # 2D plot
    def plot(channel: int):
        def wrapped(*args, **kwargs) -> sp.Expr:
            kwargs["i"] = channel
            return np.abs(np_expr(*args, **kwargs)) ** 2

        return wrapped

    for i in range(n_channels):
        iplt.plot(
            plot_domain,
            plot(i),
            ax=axes[0],
            controls=controls,
            ylim="auto",
            label=f"channel {i}",
        )
    if n_channels > 1:
        axes[0].legend(loc="upper right")
    mass_line_style = dict(
        c="red",
        alpha=0.3,
    )
    for name in controls.params:
        if not re.match(r"^m[0-9]+$", name):
            continue
        iplt.axvline(controls[name], ax=axes[0], **mass_line_style)

    # 3D plot
    color_mesh = None
    resonances_indicators = []
    threshold_indicators = []

    def plot3(*, z_cutoff, complex_rendering, **kwargs):
        nonlocal color_mesh
        Z = np_expr(plot_domain_complex, **kwargs)
        if complex_rendering == "imag":
            Z_values = Z.imag
            ax_title = "Re $T$"
        elif complex_rendering == "real":
            Z_values = Z.real
            ax_title = "Im $T$"
        elif complex_rendering == "abs":
            Z_values = np.abs(Z)
            ax_title = "$|T|$"
        else:
            raise NotImplementedError

        if n_channels == 1:
            axes[-1].set_title(ax_title)
        else:
            i = kwargs["i"]
            axes[-1].set_title(f"{ax_title}, channel {i}")

        if color_mesh is None:
            color_mesh = ax_3d.pcolormesh(X, Y, Z_values, cmap=cm.coolwarm)
        else:
            color_mesh.set_array(Z_values[:-1, :-1])
        color_mesh.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

        if resonances_indicators:
            for R, (line, text) in enumerate(resonances_indicators, 1):
                mass = kwargs[f"m{R}"]
                line.set_xdata(mass)
                text.set_x(mass + (x_max - x_min) * 0.008)
        else:
            for R in range(1, n_resonances + 1):
                mass = kwargs[f"m{R}"]
                line = ax_3d.axvline(mass, **mass_line_style)
                text = ax_3d.text(
                    x=mass + (x_max - x_min) * 0.008,
                    y=0.95 * y_min,
                    s=f"$m_{R}$",
                    c="red",
                )
                resonances_indicators.append((line, text))

        if kmatrix_type is RelativisticKMatrix:
            x_offset = (x_max - x_min) * 0.015
            if threshold_indicators:
                for i, (line_thr, line_diff, text_thr, text_diff) in enumerate(
                    threshold_indicators
                ):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = m_a - m_b
                    line_thr.set_xdata(s_thr)
                    line_diff.set_xdata(m_diff)
                    text_thr.set_x(s_thr)
                    text_diff.set_x(m_diff - x_offset)
            else:
                colors = cm.plasma(np.linspace(0, 1, n_channels))
                for i, color in enumerate(colors):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = m_a - m_b
                    line_thr = ax.axvline(s_thr, c=color, linestyle="dotted")
                    line_diff = ax.axvline(m_diff, c=color, linestyle="dashed")
                    text_thr = ax.text(
                        x=s_thr,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}+m_{{b{i}}}$",
                        c=color,
                        rotation=-90,
                    )
                    text_diff = ax.text(
                        x=m_diff - x_offset,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}-m_{{b{i}}}$",
                        c=color,
                        rotation=+90,
                    )
                    threshold_indicators.append(
                        (line_thr, line_diff, text_thr, text_diff)
                    )
            for i, (_, line_diff, _, text_diff) in enumerate(
                threshold_indicators
            ):
                m_a = kwargs[f"m_a{i}"]
                m_b = kwargs[f"m_b{i}"]
                s_thr = m_a + m_b
                m_diff = m_a - m_b
                if m_diff > x_offset + 0.01 and s_thr - abs(m_diff) > x_offset:
                    line_diff.set_alpha(0.5)
                    text_diff.set_alpha(0.5)
                else:
                    line_diff.set_alpha(0)
                    text_diff.set_alpha(0)

    # Create switch for imag/real/abs
    name = "complex_rendering"
    sliders._sliders[name] = ipywidgets.RadioButtons(
        options=["imag", "real", "abs"],
        description=R"\(s\)-plane plot",
    )
    sliders._arg_to_symbol[name] = name

    # Create cut-off slider for z-direction
    name = "z_cutoff"
    sliders._sliders[name] = ipywidgets.FloatSlider(
        value=30,
        min=+1,
        max=+100,
        step=1,
        description=R"\(z\)-cutoff",
    )
    if kmatrix_type is NonRelativisticKMatrix:
        sliders._sliders[name].value = 1
        sliders._sliders[name].min = 0.01
        sliders._sliders[name].max = 2
        sliders._sliders[name].step = 0.01
    sliders._arg_to_symbol[name] = name

    # Create GUI
    sliders_copy = dict(sliders)
    h_boxes = []
    for R in range(1, n_resonances + 1):
        buttons = [sliders_copy.pop(f"m{R}")]
        if n_channels == 1:
            if kmatrix_type is RelativisticKMatrix:
                buttons += [
                    sliders_copy.pop(symbol_to_arg[fR"\Gamma_{{{R},0}}"]),
                    sliders_copy.pop(symbol_to_arg[fR"\gamma_{{{R},0}}"]),
                ]
            if kmatrix_type is NonRelativisticKMatrix:
                buttons += [
                    sliders_copy.pop(symbol_to_arg[f"Gamma{R}"]),
                    sliders_copy.pop(symbol_to_arg[fR"\gamma_{{{R},0}}"]),
                ]
        h_box = ipywidgets.HBox(buttons)
        h_boxes.append(h_box)
    remaining_sliders = sorted(
        sliders_copy.values(), key=lambda s: (str(type(s)), s.description)
    )
    if n_channels == 1:
        remaining_sliders.remove(sliders["i"])
    ui = ipywidgets.VBox(h_boxes + remaining_sliders)
    output = ipywidgets.interactive_output(plot3, controls=sliders)
    display(ui, output)

{{ run_interactive }}

In [ ]:
plot(NonRelativisticKMatrix, n_resonances=2, n_channels=1, angular_momentum=L)